In [17]:
import re
import urllib
import warnings
from argparse import Namespace
from pathlib import Path

import torch

In [10]:
model_name = "esm2_t6_8M_UR50D"
url = f"https://dl.fbaipublicfiles.com/fair-esm/models/{model_name}.pt"
fn = Path(url).name
model_data = torch.load(
    f"{torch.hub.get_dir()}/checkpoints/{fn}",
    map_location="cpu",
)

In [12]:
def upgrade_state_dict(state_dict):
    """Removes prefixes 'model.encoder.sentence_encoder.' and 'model.encoder.'."""
    prefixes = ["encoder.sentence_encoder.", "encoder."]
    pattern = re.compile("^" + "|".join(prefixes))
    state_dict = {pattern.sub("", name): param for name, param in state_dict.items()}
    return state_dict

cfg = model_data["cfg"]["model"]
state_dict = model_data["model"]
state_dict = upgrade_state_dict(state_dict)

PATH = "../dataset/checkpoints/esm2_t6_8M_UR50D.pt"
# torch.save({
#             'cfg_model': cfg,
#             'model': state_dict,
#             }, PATH)
checkpoint = torch.load(PATH)
for k, v in checkpoint['model'].items():
    print(k, v.shape)

In [18]:
from modeling.esm.data import Alphabet
from modeling.esm.esm2 import ESM2

ModuleNotFoundError: No module named 'modeling'

In [ ]:
# import esm
# from esm.model.esm2 import ESM2
from modeling.esm.esm2 import ESM2

alphabet = esm.data.Alphabet.from_architecture("ESM-1b")

model = ESM2(
    num_layers=cfg.encoder_layers, # 6
    embed_dim=cfg.encoder_embed_dim, # 320
    attention_heads=cfg.encoder_attention_heads, # 20
    alphabet=alphabet,
    token_dropout=cfg.token_dropout, # True
)